<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-documentation/tree/master/Image/13_cumulative_cost_mapping.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-documentation/blob/master/Image/13_cumulative_cost_mapping.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-documentation/master?filepath=Image/13_cumulative_cost_mapping.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-documentation/blob/master/Image/13_cumulative_cost_mapping.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Cumulative Cost Mapping
Use `image.cumulativeCost()` to compute a cost map where every pixel contains the total cost of the lowest cost path to the nearest source location. This process is useful in a variety of contexts such as habitat analysis ([Adriaensen et al. 2003](http://www.sciencedirect.com/science/article/pii/S0169204602002426)), watershed delineation ([Melles et al. 2011](http://www.sciencedirect.com/science/article/pii/S1878029611001691)) and image segmentation ([Falcao et al. 2004](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=1261076)). Call the cumulative cost function on an image in which each pixel represents the cost per meter to traverse it. Paths are computed through any of a pixel's eight neighbors. Required inputs include a `source` image, in which each non-zero pixel represents a potential source (or start of a path), and a `maxDistance` (in meters) over which to compute paths. The algorithm finds the cumulative cost of all paths less than maxPixels = `maxDistance`/scale in length, where scale is the pixel resolution, or [scale of analysis in Earth Engine](https://developers.google.com/earth-engine/scale).

The following example demonstrates computing least-cost paths across a land cover image:

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [1]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

# Import libraries    
import ee
import folium
import geehydro

# Authenticate and initialize Earth Engine API
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [2]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [3]:
# A rectangle representing Bangui, Central African Republic.
geometry = ee.Geometry.Rectangle([18.5229, 4.3491, 18.5833, 4.4066])

# Create a source image where the geometry is 1, everything else is 0.
sources = ee.Image().toByte().paint(geometry, 1)

# Mask the sources image with itself.
sources = sources.updateMask(sources)

# The cost data is generated from classes in ESA/GLOBCOVER.
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0)

# Classes 60, 80, 110, 140 have cost 1.
# Classes 40, 90, 120, 130, 170 have cost 2.
# Classes 50, 70, 150, 160 have cost 3.
cost = \
  cover.eq(60).Or(cover.eq(80)).Or(cover.eq(110)).Or(cover.eq(140)) \
      .multiply(1).add(
  cover.eq(40).Or(cover.eq(90)).Or(cover.eq(120)).Or(cover.eq(130)) \
    .Or(cover.eq(170)) \
      .multiply(2).add(
  cover.eq(50).Or(cover.eq(70)).Or(cover.eq(150)).Or(cover.eq(160)) \
      .multiply(3)))

# Compute the cumulative cost to traverse the lAnd cover.
cumulativeCost = cost.cumulativeCost(**{
  'source': sources,
  'maxDistance': 80 * 1000 # 80 kilometers
})

# Display the results
Map.setCenter(18.71, 4.2, 9)
Map.addLayer(cover, {}, 'Globcover')
Map.addLayer(cumulativeCost, {'min': 0, 'max': 5e4}, 'accumulated cost')
Map.addLayer(geometry, {'color': 'FF0000'}, 'source geometry')

The result should look something like Figure 1, in which each output pixel represents the accumulated cost to the nearest source. Note that discontinuities can appear in places where the least cost path to the nearest source exceeds maxPixels in length.

## Display Earth Engine data layers 

In [4]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map